# Modelling

Este notebook é dedicado exclusivamente a parte de modelagem e avaliação dos respectivos modelos testados. No final deste arquivo encontra-se a previsão do preço para novos dados de entrada e também o processo de exportação do modelo já definido em um arquivo .pkl conforme solicitado pelo desafio.

## Entendendo o problema

Antes de inicar o desenvolvimento, primeiro é preciso entender de que tipo de problema de machine learning estamos falando. Como a ideia aqui é prever preços de aluguel, ou seja, um NÚMERO, um valor contínuo, portanto se trata de um problema de REGRESSÃO. Caso a ideia fosse prever alguma categoria, ou grupo, sendo este um dado discreto, ai sim estaríamos falando de um problema de classificação.

Além disso, sabe-se que se trata de um modelo SUPERVISIONADO, ou seja, nós temos descriminados em nosso dataset de treino e teste a variável que se deseja prever ('price'). Tem-se um amostral deste "target". Se por um acaso o "target" não estivesse descriminado ou definido, forçando com que o algoritmo tivesse que extrair um padrão de todo o amostral X, então se trataria de um modelo NÃO SUPERVISIONADO. Ai entrariam modelos de clusterização, recomendação e assim por diante.

Entendido isso, pode-se então partir para o desenvolvimento utilizando algoritmos e estratégias para resolver um problema de regressão.

## Importando as bibliotecas

Para iniciar o desenvolvimento, primeiro importou-se as bibliotecas que serão utilizadas nesta etapa de modelling,

In [4]:
# lib para salvar no formato .pkl
import pickle

# lib de manipulação dos dados
import pandas as pd
import numpy as np

# pré-processamento de dados
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# separação das colunas por tipo de dado
from sklearn.compose import make_column_selector

# pipeline para tratamento dos dados
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# seleção e validação de modelos
from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV

# modelos de Regressão a serem testados
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor

# manter a saída do processamento do sklearn como DataFrame pandas
from sklearn import set_config
set_config(transform_output = 'pandas')

## Definindo as métricas de avaliação para os modelos de regressão a serem testados

Quando se fala em modelos de regressão, existem algumas métricas importantes para estas avaliações. Segue abaixo uma breve análise de 4 das mais importantes e famosas neste tipo de problema.

- R² = também chamado de coeficiente de determinação, é uma métrica que expressa a porcentagem da variância dos dados que pode ser explicada pelo modelo. É uma métrica que varia de 0 até 1, sendo quanto mais próximo de 1 maior a capacidade do modelo em explicar as variações dos dados.
- MAE = erro médio absoluto, basicamente mede a distância média entre os valores previstos pelo modelo e os valores reais. Outro ponto de destaque é que a unidade deste erro é a mesma do 'target', ou seja, tem a mesma unidade da variável preço. Contudo, esta métrica é muito sensível a outliers, desta forma, a tendência é "puxar" o modelo mais próximo dos outliers, deixando-o mais distante da maior concentração dos dados reais. Conforme foi visto nas análises feitas no notebook EDA, este dataset tem forte presença de outliers, portanto o MAE não seria o ideal para avaliar os modelos.
- MSE = erro médio quadrático, basicamente é a diferença entre o valor previsto e o valor real, elevando-se o resultado ao quadrado. Da mesma forma que o MAE, quanto maior esta métrica pior será o modelo. Como neste caso os erros são elevados ao quadrado, isto torna esta métrica muito boa para casos em que grande erros não são tolerados, deixando-a muito menos sensível a outliers, se comparada ao MAE.
- RMSE = raiz do erro quadrático médio, basicamente é a métrica MSE tirando a sua raiz quadrada. Desta forma, tem-se um valor de melhor interpretabilidade, corringindo também as unidades deste erro.

Considerando as características expostas acima e também as características dos dados deste dataset, expostas no notebook 'EDA.ipynb', optou-se por avaliar os modelos testados utilizando as métricas MSE e RMSE, visto que trabalham melhor em casos de outlier, sendo menos tolerantes com erros muito grandes. Além disso optou-se por expor junto a métrica R² por conta da sua fácil interpretabilidade.

Escolhidas as métricas utilizadas para a avaliação dos modelos, optou-se  por definir a função abaixo, que será chamada todas as vezes que for necessário calcular as métricas de algum modelo, a fim de deixar o código posteriormente mais "limpo".

In [8]:
# função para calcular as métricas dos modelos
def get_metrics (y_true, y_pred):
    dict_metrics = {
        'R2': metrics.r2_score(y_true, y_pred),
        'MSE': metrics.mean_squared_error(y_true, y_pred),
        'RMSE': np.sqrt(metrics.mean_squared_error(y_true, y_pred))
    }
    return dict_metrics